<a href="https://colab.research.google.com/github/chenf79/test/blob/master/pyod_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install pyod
# !pip install combo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.0/150.0 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyod: filename=pyod-1.0.9-py3-none-any.whl size=184097 sha256=80e9c92a84a03d0c6bf5f4ea2b86359f5113d4f0d070120fde4ee43b1b49167b
  Stored in directory: /root/.cache/pip/wheels/83/55/6b/552e083cf5509c0afe808b76cf434f1be284d01a112623bd37
Successfully built pyod


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for combo: filename=combo-0.1.3-py3-none-any.whl size=42865 sha256=cf08bbcb75cdad1aacd02d857dd5df3905b12c620ab816b8cf593c009d4b654a
  Stored in directory: /root/.cache/pip/wheels/7a/48/ff/0d6e1017d8bf3d844d332736fbb268dca767162779f93ff50a
Successfully built combo


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# -*- coding: utf-8 -*-
"""Benchmark of all implemented algorithms
"""
# Author: Yue Zhao <zhaoy@cmu.edu>
# License: BSD 2 clause

from __future__ import division
from __future__ import print_function

import os
import sys
from time import time

# temporary solution for relative imports in case pyod is not installed
# if pyod is installed, no need to use the following line
sys.path.append(
    os.path.abspath(os.path.join(os.path.dirname("__file__"), '..')))
# supress warnings for clean output
import warnings

warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.io import loadmat

from pyod.models.abod import ABOD
from pyod.models.cblof import CBLOF
from pyod.models.feature_bagging import FeatureBagging
from pyod.models.hbos import HBOS
from pyod.models.iforest import IForest
from pyod.models.knn import KNN
from pyod.models.lof import LOF
from pyod.models.mcd import MCD
from pyod.models.ocsvm import OCSVM
from pyod.models.pca import PCA
from pyod.models.cof import COF
from pyod.models.sod import SOD

from pyod.utils.utility import standardizer
from pyod.utils.utility import precision_n_scores
from sklearn.metrics import roc_auc_score


In [ ]:

# Implement a function that takes a X_train and X_test, as INPUT, and returns method_anomalies 
# as OUTPUT.  Method_anomalies is a list of 9 lists corresponding to the different 
# anomalies detected by the nine AD methods (LOF, KNN, CBLOF, SOD, IForest, OCSVM, 
# HBOS, PCA, COPOD). For example, Method_anomalies[0] is a list of anomalies detected 
# by the first method LOF. Each anomaly is represented by the data point index and 
# the associated anomaly score. 
# method_anomalies[0][k] stores [data point index, anomaly score] of the k-th anomaly detected by LOF.

def multi_ad_detectors(X_train, X_test):

  method_anomalies = []
  classifiers = {'Angle-based Outlier Detector (ABOD)': ABOD(
      contamination=outliers_fraction),
      'Cluster-based Local Outlier Factor': CBLOF(
          n_clusters=10,
          contamination=outliers_fraction,
          check_estimator=False,
          random_state=random_state),
      'Feature Bagging': FeatureBagging(contamination=outliers_fraction,
                                        random_state=random_state),
      'Histogram-base Outlier Detection (HBOS)': HBOS(
          contamination=outliers_fraction),
      'Isolation Forest': IForest(contamination=outliers_fraction,
                                  random_state=random_state),
      'K Nearest Neighbors (KNN)': KNN(contamination=outliers_fraction),
      'Local Outlier Factor (LOF)': LOF(
          contamination=outliers_fraction),
      'Minimum Covariance Determinant (MCD)': MCD(
          contamination=outliers_fraction, random_state=random_state),
      'One-class SVM (OCSVM)': OCSVM(contamination=outliers_fraction),
      'Principal Component Analysis (PCA)': PCA(
          contamination=outliers_fraction, random_state=random_state),
  }
  classifiers_indices = {
      'Angle-based Outlier Detector (ABOD)': 0,
      'Cluster-based Local Outlier Factor': 1,
      'Feature Bagging': 2,
      'Histogram-base Outlier Detection (HBOS)': 3,
      'Isolation Forest': 4,
      'K Nearest Neighbors (KNN)': 5,
      'Local Outlier Factor (LOF)': 6,
      'Minimum Covariance Determinant (MCD)': 7,
      'One-class SVM (OCSVM)': 8,
      'Principal Component Analysis (PCA)': 9,
  }

  for clf_name, clf in classifiers.items():
      t0 = time()
      clf.fit(X_train)
      test_scores = clf.decision_function(X_test)
      t1 = time()
      duration = round(t1 - t0, ndigits=4)

      roc = round(roc_auc_score(y_test, test_scores), ndigits=4)
      prn = round(precision_n_scores(y_test, test_scores), ndigits=4)

      print('{clf_name} ROC:{roc}, precision @ rank n:{prn}, '
            'execution time: {duration}s'.format(
          clf_name=clf_name, roc=roc, prn=prn, duration=duration))
        
  return method_anomalies

In [17]:


# Define data file and read X and y
mat_file_list = ['arrhythmia.mat',
                 'cardio.mat',
                 'glass.mat',
                 'ionosphere.mat',
                 'letter.mat',
                 'lympho.mat',
                 'mnist.mat',
                 'musk.mat',
                 'optdigits.mat',
                 'pendigits.mat',
                 'pima.mat',
                 'satellite.mat',
                 'satimage-2.mat',
                 'shuttle.mat',
                 'vertebral.mat',
                 'vowels.mat',
                 'wbc.mat']

# define the number of iterations
n_ite = 10
n_classifiers = 10

df_columns = ['Data', '#Samples', '# Dimensions', 'Outlier Perc',
              'ABOD', 'CBLOF', 'FB', 'HBOS', 'IForest', 'KNN', 'LOF',
              'MCD', 'OCSVM', 'PCA']


# initialize the container for saving the results
roc_df = pd.DataFrame(columns=df_columns)
prn_df = pd.DataFrame(columns=df_columns)
time_df = pd.DataFrame(columns=df_columns)

for j in range(len(mat_file_list))[:1]:

    mat_file = mat_file_list[j]
    mat = loadmat(os.path.join('/content/drive/MyDrive/datasets/', mat_file))

    X = mat['X']
    y = mat['y'].ravel()
    outliers_fraction = np.count_nonzero(y) / len(y)
    outliers_percentage = round(outliers_fraction * 100, ndigits=4)

    # construct containers for saving results
    roc_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
    prn_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
    time_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]

    roc_mat = np.zeros([n_ite, n_classifiers])
    prn_mat = np.zeros([n_ite, n_classifiers])
    time_mat = np.zeros([n_ite, n_classifiers])

    for i in range(n_ite):
        print("\n... Processing", mat_file, '...', 'Iteration', i + 1)
        random_state = np.random.RandomState(i)

        # 60% data for training and 40% for testing
        X_train, X_test, y_train, y_test = \
            train_test_split(X, y, test_size=0.4, random_state=random_state)

        # standardizing data for processing
        X_train_norm, X_test_norm = standardizer(X_train, X_test)

        multi_ad_detectors(X_train_norm, X_test_norm)




... Processing arrhythmia.mat ... Iteration 1
Angle-based Outlier Detector (ABOD) ROC:0.8035, precision @ rank n:0.3462, execution time: 0.361s
Cluster-based Local Outlier Factor ROC:0.7995, precision @ rank n:0.4615, execution time: 2.6579s
Feature Bagging ROC:0.8094, precision @ rank n:0.5385, execution time: 1.6231s
Histogram-base Outlier Detection (HBOS) ROC:0.8608, precision @ rank n:0.5385, execution time: 0.2297s
Isolation Forest ROC:0.8476, precision @ rank n:0.5385, execution time: 0.7531s
K Nearest Neighbors (KNN) ROC:0.8176, precision @ rank n:0.5, execution time: 0.2733s
Local Outlier Factor (LOF) ROC:0.8082, precision @ rank n:0.5, execution time: 0.1692s
Minimum Covariance Determinant (MCD) ROC:0.8464, precision @ rank n:0.5, execution time: 3.7004s
One-class SVM (OCSVM) ROC:0.8102, precision @ rank n:0.5, execution time: 0.0818s
Principal Component Analysis (PCA) ROC:0.8164, precision @ rank n:0.5, execution time: 0.1884s

... Processing arrhythmia.mat ... Iteration 2
A

Mounted at /content/drive


# New Section

# New Section